In [ ]:
! pip install kaggle

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


In [ ]:
#https://www.kaggle.com/code/bettycxh06/rri-amplitudes/input
!kaggle datasets download -d bettycxh06/apnea-ecg-database

!kaggle kernels pull bettycxh06/rri-amplitudes


 95% 300M/315M [00:02<00:00, 143MB/s]
100% 315M/315M [00:03<00:00, 107MB/s]
Source code downloaded to /content/rri-amplitudes.ipynb


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
os.listdir(os.getcwd())

['.config',
 'kaggle.json',
 'rri-amplitudes.ipynb',
 'apnea-ecg-database.zip',
 'sample_data']

In [ ]:
!unzip apnea-ecg-database.zip

Archive:  apnea-ecg-database.zip
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/ANNOTATORS  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/RECORDS  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/SHA256SUMS.txt  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.apn  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.dat  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.hea  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.qrs  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.xws  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.apn  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.hea  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.qrs  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.xws  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01r.apn  

In [ ]:
!pip install biosppy
!pip install wfdb

from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import posixpath
import csv
import wfdb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 3.6 MB/s eta 0:00:00


In [ ]:
import pickle
import sys
from concurrent.futures import ProcessPoolExecutor, as_completed

import biosppy.signals.tools as st
import numpy as np
import os
import wfdb
from biosppy.signals.ecg import correct_rpeaks, hamilton_segmenter
from scipy.signal import medfilt
from tqdm import tqdm

# PhysioNet Apnea-ECG dataset
# url: https://physionet.org/physiobank/database/apnea-ecg/
#base_dir = "../input/apnea-ecg-database/apnea-ecg-database-1.0.0"
base_dir = "apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0"
#base_dir = "physionet.org/files/apnea-ecg/1.0.0"



fs = 100
sample = fs * 60  # 1 min's sample points

before = 2  # forward interval (min)
after = 2  # backward interval (min)
hr_min = 20
hr_max = 300

num_worker = 35


def worker(name, labels):
    X = []
    y = []
    groups = []
    signals = wfdb.rdrecord(os.path.join(base_dir, name), channels=[0]).p_signal[:, 0]
    for j in tqdm(range(len(labels)), desc=name, file=sys.stdout):
        if j < before or \
                (j + 1 + after) > len(signals) / float(sample):
            continue
        signal = signals[int((j - before) * sample):int((j + 1 + after) * sample)]
        signal, _, _ = st.filter_signal(signal, ftype='FIR', band='bandpass', order=int(0.3 * fs),
                                        frequency=[3, 45], sampling_rate=fs)
        # Find R peaks
        rpeaks, = hamilton_segmenter(signal, sampling_rate=fs)
        rpeaks, = correct_rpeaks(signal, rpeaks=rpeaks, sampling_rate=fs, tol=0.1)
        if len(rpeaks) / (1 + after + before) < 40 or \
                len(rpeaks) / (1 + after + before) > 200:  # Remove abnormal R peaks signal
            continue
        # Extract RRI, Ampl signal
        rri_tm, rri_signal = rpeaks[1:] / float(fs), np.diff(rpeaks) / float(fs)
        rri_signal = medfilt(rri_signal, kernel_size=3)
        ampl_tm, ampl_siganl = rpeaks / float(fs), signal[rpeaks]
        hr = 60 / rri_signal
        # Remove physiologically impossible HR signal
        if np.all(np.logical_and(hr >= hr_min, hr <= hr_max)):
            # Save extracted signal
            X.append([(rri_tm, rri_signal), (ampl_tm, ampl_siganl)])
            y.append(0. if labels[j] == 'N' else 1.)
            groups.append(name)
    return X, y, groups


if __name__ == "__main__":
    apnea_ecg = {}

    names = [
        "a01", "a02", "a03", "a04", "a05", "a06", "a07", "a08", "a09", "a10",
        "a11", "a12", "a13", "a14", "a15", "a16", "a17", "a18", "a19", "a20",
        "b01", "b02", "b03", "b04", "b05",
        "c01", "c02", "c03", "c04", "c05", "c06", "c07", "c08", "c09", "c10"
    ]

    o_train = []
    y_train = []
    groups_train = []
    print('Training...')
    with ProcessPoolExecutor(max_workers=num_worker) as executor:
        task_list = []
        for i in range(len(names)):
            labels = wfdb.rdann(os.path.join(base_dir, names[i]), extension="apn").symbol
            task_list.append(executor.submit(worker, names[i], labels))

        for task in as_completed(task_list):
            X, y, groups = task.result()
            o_train.extend(X)
            y_train.extend(y)
            groups_train.extend(groups)

    print()



    answers = {}
    with open(os.path.join("event-2-answers"), "r") as f:
    #    with open(os.path.join(base_dir, "event-2-answers"), "r") as f:
        for answer in f.read().split("\n\n"):
            answers[answer[:3]] = list("".join(answer.split()[2::2]))

    names = [
        "x01", "x02", "x03", "x04", "x05", "x06", "x07", "x08", "x09", "x10",
        "x11", "x12", "x13", "x14", "x15", "x16", "x17", "x18", "x19", "x20",
        "x21", "x22", "x23", "x24", "x25", "x26", "x27", "x28", "x29", "x30",
        "x31", "x32", "x33", "x34", "x35"
    ]

    o_test = []
    y_test = []
    groups_test = []
    print("Testing...")
    with ProcessPoolExecutor(max_workers=num_worker) as executor:

        task_list = []
        for i in range(len(names)):
            labels = answers[names[i]]
            task_list.append(executor.submit(worker, names[i], labels))

        for task in as_completed(task_list):
            X, y, groups = task.result()
            o_test.extend(X)
            y_test.extend(y)
            groups_test.extend(groups)

    apnea_ecg = dict(o_train=o_train, y_train=y_train, groups_train=groups_train, o_test=o_test, y_test=y_test,
                     groups_test=groups_test)
    with open(os.path.join(base_dir, "apnea-ecg.pkl"), "wb") as f:
        pickle.dump(apnea_ecg, f, protocol=2)

    print("\nok!")

Training...
a17: 100%|██████████| 485/485 [48:08<00:00,  5.96s/it]

Testing...
x12: 100%|██████████| 527/527 [49:03<00:00,  5.59s/it]

ok!


In [ ]:
!pip install keras
!pip install tensorflow
#
#IMPORT LIBRARIES
import pickle
import numpy as np
import tensorflow
import os
from keras.callbacks import LearningRateScheduler,EarlyStopping
from keras.layers import Dense,Flatten,MaxPooling2D,Conv2D,BatchNormalization
from keras.regularizers import l2
from scipy.interpolate import splev, splrep
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
import keras
import matplotlib.pyplot as plt
from keras.layers import Conv1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.models import Model, load_model, save_model
from keras.layers import Input
import pandas as pd

#base_dir = "dataset"
base_dir = "apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0"
#base_dir = "physionet.org/files/apnea-ecg/1.0.0"

#------------------------------------------------------------------------------
# DEEP LEARNING MODELS NEED TO VECTORS OR MATRIX WITH SAME SIZE
# R-R INTERVALS DONT HAVE SAME SIZE, SO WE NEED TO INTERPOLATE VECTORS TO GET VECTORS WITH SAME SIZE.
# BASED ON OUR EXPERIENCE INTERPOLATION IN 3 HZ BETTER AND ACCURATE.
ir = 3 # INTERPOLATION RATE(3HZ)
time_range= 60 # 60-s INTERVALS OF ECG SIGNALS
weight=1e-3 #  WEIGHT L2 FOR REGULARIZATION(AVODING OVERFITTING PARAMETER)
#-----------------------------
# NORMALIZATION:
# DEEP LEARNING AND EVEN NEURAL NETWORKS INPUT SHOULD BE NORMALIZED:
# MIN-MAX METHOD APPLIED FOR SCALING:(Array-min(Array))/(max(Array)-min(Array))
scaler = lambda arr: (arr - np.min(arr)) / (np.max(arr) - np.min(arr))
#-----------------------------
# FIRSTLY WE PRE-PROCESSED OUR DATA IN "apnea-ecg.pkl" FILE
# IN PRE-PROCESSING SECTION WE EXTRACT R-R INTERVALS AND R-PEAK AMPLITUDES
# IN THIS PART WE LOAD THIS DATA AND INTERPOLATE AND CONCATE FOR FEEDING TO NETWORKS
def load_data():
    tm = np.arange(0, (before + 1 + after) * 60, step=1 / float(ir))

    with open(os.path.join(base_dir, "apnea-ecg.pkl"), 'rb') as f: # read preprocessing result
        apnea_ecg = pickle.load(f)
#-----------------
    x_train = []
    o_train, y_train = apnea_ecg["o_train"], apnea_ecg["y_train"]

    for i in range(len(o_train)):
        (rri_tm, rri_signal), (amp_tm, amp_signal) = o_train[i]
      # Curve interpolation
        rri_interp_signal = splev(tm, splrep(rri_tm, scaler(rri_signal), k=3), ext=1)
        amp_interp_signal = splev(tm, splrep(amp_tm, scaler(amp_signal), k=3), ext=1)
        x_train.append([rri_interp_signal, amp_interp_signal])
    x_train = np.array(x_train, dtype="float32")

    x_train = np.expand_dims(x_train,1)
    x_train=np.array(x_train, dtype="float32").transpose((0,3,1,2)) # convert to numpy format

    #return x_train_final, y_train
    return x_train, y_train

In [ ]:
!pip install Hyperactive
!pip install mealpy

from keras.layers.rnn import GRU
from keras.layers import Reshape, Lambda
from keras.layers import GRU
from hyperactive import Hyperactive
from hyperactive.optimizers import HillClimbingOptimizer
from hyperactive.optimizers import RandomSearchOptimizer
from mealpy.swarm_based.GWO import OriginalGWO

from tensorflow.keras import optimizers
#from Hyperactive.hyperactive import RandomSearchOptimizer, MHoneyBadgerAlgorithm, HoneyBadgerAlgorithm
from keras.optimizers import adam
import threading
import concurrent.futures
from threading import Thread
import time
from tabnanny import verbose
from mealpy.swarm_based.HBA import OriginalHBA

from tensorflow.python.keras import optimizers
#from keras.optimizers import RandomSearchOptimizer

import warnings
warnings.filterwarnings('ignore')


In [ ]:
#print(x_train.shape, y_train.shape)
#x_train= np.reshape(x_train,(x_train.shape[0],450,2,2))    #90---450?
#x_train= np.concatenate((x_train[:,:450,:,0],x_train[:,:450,:,1]),axis=1)
#x_train= x_train.transpose((0,2,1))
#print(x_train.shape, y_train.shape)
##(16709, 900, 1, 2) (16709, 2)
##(16709, 2, 900) (16709, 2)


In [ ]:
# BHA
###########################
def fitness_function(position):
    learning_rate,n1,n2,n3,n4,n5,n6,epochs = position[0],position[1],position[2],position[3],position[4],position[5],position[6],position[7]
    n1,n2,n3,n4,n5,n6,epochs=int(n1),int(n2),int(n3),int(n4),int(n5),int(n6),int(epochs)
    #return learning_rate,n1,n2,n3,n4,n5,n6,epochs

    print(learning_rate,n1,n2,n3,n4,n5,n6,epochs)

#def create_model(weight=1e-3):
    model= Sequential()
    model.add(Conv2D(n1, kernel_size=(11,1), strides=(1,1), padding="valid", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight),input_shape=(900,1,2)))    #,input_shape=(180,1,2)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3,1)))
    model.add(Conv2D(n2, kernel_size=(5,1), strides=1, padding="valid", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3,1)))
    model.add(Conv2D(n3, kernel_size=(3,1), strides=(1,1), padding="valid", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(BatchNormalization())
    model.add(Conv2D(n4, kernel_size=(3,1), strides=1, padding="valid", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(BatchNormalization())
    model.add(Conv2D(n5, kernel_size=(3,1), strides=1, padding="valid", activation="relu", kernel_initializer="he_normal",
                kernel_regularizer=l2(weight), bias_regularizer=l2(weight)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3,1),strides=(2,1)))

    model.add(Flatten())
    model.add(Dense(n6, activation="relu"))
    model.add(Dense(2, activation="softmax"))

    return learning_rate,n1,n2,n3,n4,n5,n6,epochs


    optimizer =keras.optimizers.Adam(lr=learning_rate)
    print("Learning rate: ", learning_rate)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
    #model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=['accuracy'])

    history=model.fit(X1, Y1, batch_size=128, epochs=int(epochs), validation_data=(x_val, y_val),
                       callbacks=[callback1,lr_scheduler])


    #history = model.fit(x_train,y_train, epochs=int(epochs),batch_size=32, validation_data=(x_test, y_test))
    return history.history['val_accuracy'][-1]
    #loss, accuracy = model.evaluate(x_test, y_test)

    #return model
#------------------------------------------------------------------------------
# Define learning rate schedule for preventing overfitting in deep learning methods:
def lr_schedule(epochs, learning_rate):
   if epochs > 70 and \
           (epochs - 1) % 10 == 0:
        learning_rate *= 0.1
   print("Learning rate: ", learning_rate)
   return learning_rate

#------------------------------------------------------------------------------

        # we used k-fold cross-validation for more reliable experiments:
   kfold = StratifiedKFold(n_splits=5, shuffle=True,random_state=7)
   cvscores = []
   ACC=[]
   SN=[]
   SP=[]
   F2=[]
   print("train num:", len(y_train))
   print("test num:", len(y_test))


    # separate train& test and then compile model
  # for train, test in kfold.split(x_train, y_train.argmax(1)):
     #model = create_model()
     #model.summary()

     # Compile and evaluate model:
if __name__ == "__main__":
    # loading Data:
    x_train, y_train = load_data()
    # we have labels(Y) in a binary way 0 for normal and 1 for apnea patients
    # we want to classify data into 2-class so we changed y in a categorical way:
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
    # we used k-fold cross-validation for more reliable experiments:
    kfold = StratifiedKFold(n_splits=5, shuffle=True,random_state=7)


    # separate train& test and then compile model
    ##for train, test in kfold.split(x_train, y_train.argmax(1)):
     #model = create_model()
     #model.summary()

    # define callback for early stopping:
    lr_scheduler = LearningRateScheduler(lr_schedule)
    callback1 = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    x_train= np.reshape(x_train,(x_train.shape[0],450,2,2))    #90---450?
    x_train= np.concatenate((x_train[:,:450,:,0],x_train[:,:450,:,1]),axis=1)
    x_train= x_train.transpose((0,2,1))
    #10% of Data used for validation:
    X1,x_val,Y1,y_val=train_test_split(x_train,y_train,test_size=0.10)

##################################################################
#pso = Pso(swarmsize=4,maxiter=14)
# n,sf,sp,l
##bp,value = pso.run(func,[1,2,2,2],[16,8,4,4])
#v = func(bp);
##################################################################
problem_dict1 = {
    "fit_func": fitness_function,
    'lb':[0.001,86,156,284,284,156,109,30], # Lower bound of our parameters
    'ub':[0.001,120,356,494,494,356,309,150], # upper bound of our parameters
    "minmax": "min",
    "obj_weights": [0.4,1,1,1,1,1,1,1]               # Define it or default value will be [1, 1, 1]   [0.4, 0.1, 0.5,1,1,1,1,1]
    #"mode":"thread"
    #"amend_position": amend_position

                }

rng = np.random.default_rng()
epoch = 100
pop_size =50

#def thread_function(k):

  #run=k+1
  #print("Run Number",run)
  #Optimizer = MHoneyBadgerAlgorithm(search_config, n_iter=100, n_part=10, metric='accuracy', cv=10, h_beta=6.0, h_c=2.0,run=run)
model = OriginalGWO(epoch, pop_size)
  #t1 = time.time()

  #Optimizer.fit(x_train, y_train)
 # t2 = time.time()

  #print("time: {}".format(t2-t1))



#best_position, best_fitness = model.solve(problem_dict1)
best_position, best_fitness= model.solve(problem_dict1)
#best_position, best_fitness= model.solve(mode="thread")
v = fitness_function(best_position);
#print(f"Solution: {best_position}, Fitness: {best_fitness}")
#return v
print(f"Solution: {best_position}, Fitness: {v}")

INFO:mealpy.swarm_based.GWO.OriginalGWO:Solving 8-objective optimization problem with weights: [0.4 1.  1.  1.  1.  1.  1.  1. ].


0.001 99 280 400 392 355 222 107
0.001 90 179 343 367 340 300 135
0.001 102 202 437 373 306 224 70
0.001 110 224 292 404 191 250 85
0.001 89 249 479 437 307 217 117
0.001 119 178 474 362 220 172 34
0.001 118 159 437 387 330 146 43
0.001 97 157 395 327 212 142 77
0.001 111 161 332 459 233 157 68
0.001 101 287 327 451 305 221 85
0.001 93 217 409 458 252 256 54
0.001 91 319 362 445 210 295 131
0.001 94 176 468 370 274 209 57
0.001 111 171 340 371 182 220 82
0.001 87 226 410 452 317 177 51
0.001 97 301 468 459 259 138 84
0.001 110 308 323 419 174 125 101
0.001 117 164 491 405 307 258 39
0.001 112 169 418 406 174 269 72
0.001 95 294 478 448 251 239 135
0.001 96 270 332 429 217 285 124
0.001 88 195 441 318 184 132 50
0.001 86 340 446 461 309 207 39
0.001 93 231 386 331 198 191 79
0.001 100 205 421 303 348 269 102
0.001 89 175 458 377 248 246 86
0.001 116 331 425 421 330 166 106
0.001 89 187 444 338 169 237 80
0.001 94 257 426 449 255 114 144
0.001 101 287 363 452 264 183 67
0.001 87 200 301 

INFO:mealpy.swarm_based.GWO.OriginalGWO:>Problem: P, Epoch: 1, Current best: 1177.0004, Global best: 1177.0004, Runtime: 7.43569 seconds


0.001 114 156 410 284 222 116 30
0.001 86 179 379 284 297 142 30
0.001 120 190 477 484 262 292 60
0.001 118 156 376 494 356 109 30
0.001 117 156 284 284 196 146 30
0.001 120 181 494 316 356 135 45
0.001 86 156 317 442 271 165 30
0.001 86 157 438 298 156 136 38
0.001 120 190 284 284 156 109 30
0.001 86 160 284 451 300 128 30
0.001 95 270 494 284 156 140 50
0.001 86 156 284 425 242 109 30
0.001 89 156 284 284 156 206 95
0.001 86 156 400 284 156 179 59
0.001 86 156 494 284 234 192 51
0.001 86 156 288 293 268 109 53
0.001 86 156 284 284 156 109 34
0.001 105 156 304 363 180 204 30
0.001 120 356 284 284 269 171 30
0.001 86 156 284 284 184 179 33
0.001 86 156 342 419 255 145 30
0.001 86 156 336 379 215 138 40
0.001 95 229 284 422 156 109 30
0.001 86 156 494 284 231 126 30
0.001 120 156 284 284 306 109 31
0.001 98 225 425 368 319 147 43
0.001 120 167 284 284 156 187 43
0.001 86 200 494 284 203 309 30
0.001 86 183 284 284 229 157 55
0.001 105 156 473 284 156 187 30
0.001 101 215 494 284 282 109

INFO:mealpy.swarm_based.GWO.OriginalGWO:>Problem: P, Epoch: 2, Current best: 1107.0004, Global best: 1107.0004, Runtime: 7.99798 seconds


...

0.001 86 156 284 284 156 109 30
0.001 86 156 284 286 156 109 30
0.001 86 157 284 284 156 109 30
0.001 86 156 284 284 157 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 285 284 156 109 30
0.001 86 156 285 286 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 286 284 157 109 30
0.001 86 157 285 284 157 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 157 109 30
0.001 86 156 284 285 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 110 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 285 285 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 285 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 157 109 30
0.001 86 156 284 284 157 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86

INFO:mealpy.swarm_based.GWO.OriginalGWO:>Problem: P, Epoch: 99, Current best: 1105.0004, Global best: 1105.0004, Runtime: 6.72843 seconds


0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
0.001 86

INFO:mealpy.swarm_based.GWO.OriginalGWO:>Problem: P, Epoch: 100, Current best: 1105.0004, Global best: 1105.0004, Runtime: 7.58328 seconds


0.001 86 156 284 284 156 109 30
0.001 86 156 284 284 156 109 30
Solution: [1.00e-03 8.60e+01 1.56e+02 2.84e+02 2.84e+02 1.56e+02 1.09e+02 3.00e+01], Fitness: (0.001, 86, 156, 284, 284, 156, 109, 30)
